In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [2]:
# load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
len(train)

38932

In [4]:
len(test)

29404

In [5]:
train[1:5].as_matrix

<bound method DataFrame.as_matrix of    User_ID                                        Description  \
1  id10327  I stayed at the Crown Plaza April -- - April -...   
2  id10328  I booked this hotel through Hotwire at the low...   
3  id10329  Stayed here with husband and sons on the way t...   
4  id10330  My girlfriends and I stayed here to celebrate ...   

        Browser_Used Device_Used Is_Response  
1  Internet Explorer      Mobile   not happy  
2            Mozilla      Tablet   not happy  
3   InternetExplorer     Desktop       happy  
4               Edge      Tablet   not happy  >

In [6]:
train['Description'][1] 

"I stayed at the Crown Plaza April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but priced a little on the high side. Of course this is Washington DC. There is no pool and little for children to do. My room on the fifth floor had two comfortable beds and plenty of space for one person. The TV is a little small by todays standards with a limited number of channels. There was a small bit of mold in the bathtub area that could have been removed with a little bleach. It appeared the carpets were not vacummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but the one on my floor did not work. I encountered some staff in the elevator one evening and I mentioned the ice machine to them. Severel hours later a maid appeared at my door with ice and two mints. I'm not sure how they knew what room I was in. That was a little unnervi

In [7]:
train['Description'][1] 

"I stayed at the Crown Plaza April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but priced a little on the high side. Of course this is Washington DC. There is no pool and little for children to do. My room on the fifth floor had two comfortable beds and plenty of space for one person. The TV is a little small by todays standards with a limited number of channels. There was a small bit of mold in the bathtub area that could have been removed with a little bleach. It appeared the carpets were not vacummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but the one on my floor did not work. I encountered some staff in the elevator one evening and I mentioned the ice machine to them. Severel hours later a maid appeared at my door with ice and two mints. I'm not sure how they knew what room I was in. That was a little unnervi

In [8]:
# print stopwords.words('english')
print set(stopwords.words('english'))

set([u'all', u'just', u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u'do', u'them', u'his', u'very', u'they', u'not', u'during', u'now', u'him', u'nor', u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u'where', u'few', u'because', u'doing', u'some', u'hasn', u'are', u'our', u'ourselves', u'out', u'what', u'for', u'while', u're', u'does', u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u'were', u'here', u'shouldn', u'hers', u'by', u'on', u'about', u'couldn', u'of', u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u'mightn', u'wasn', u'your', u'from', u'her', u'their', u'aren', u'there', u'been', u'whom', u'too', u'wouldn', u'themselves', u'weren', u'was', u'until', u'more', u'himself', u'that', u'but', u'don', u'with', u'than', u'those', u'he', u'me', u'myself', u'ma', u'these', u'up', u'will', u'below', u'ain

In [120]:
stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
#     if stemming:
#         st = PorterStemmer()
#         txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [121]:
test['Is_Response'] = np.nan
alldata = pd.concat([train, test]).reset_index(drop=True)

In [122]:
print type(alldata['Description'])

<class 'pandas.core.series.Series'>


In [123]:
alldata['Description'] = alldata['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

In [124]:
alldata['Description'][1]

'stayed crown plaza april april staff friendly attentive elevators tiny food restaurant delicious priced little high side course washington dc pool little children room fifth floor two comfortable beds plenty space one person tv little small todays standards limited number channels small bit mold bathtub area could removed little bleach appeared carpets vacummed every day reported light bulb burned never replaced ice machines odd numbered floors one floor work encountered staff elevator one evening mentioned ice machine severel hours later maid appeared door ice two mints im sure knew room little unnerving would stay business would come vacation'

In [125]:
alldata['Description'][1]

'stayed crown plaza april april staff friendly attentive elevators tiny food restaurant delicious priced little high side course washington dc pool little children room fifth floor two comfortable beds plenty space one person tv little small todays standards limited number channels small bit mold bathtub area could removed little bleach appeared carpets vacummed every day reported light bulb burned never replaced ice machines odd numbered floors one floor work encountered staff elevator one evening mentioned ice machine severel hours later maid appeared door ice two mints im sure knew room little unnerving would stay business would come vacation'

In [126]:
for_w2v = alldata['Description'].apply(lambda x:x.split())
for_w2v  

for_w2v_train = for_w2v[0:len(train)]
(for_w2v_train)
for_w2v_test = for_w2v[len(train):]
len(for_w2v_test)

29404

In [127]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec

print "Training model..."
model = word2vec.Word2Vec(for_w2v, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


Training model...


In [128]:
 model.most_similar("clean")

[('spotless', 0.6777294874191284),
 ('immaculate', 0.6248620748519897),
 ('surprisingly', 0.4992624521255493),
 ('wellappointed', 0.4830130636692047),
 ('adequate', 0.4722539782524109),
 ('smallish', 0.4710160791873932),
 ('roomy', 0.47007885575294495),
 ('functional', 0.4435805082321167),
 ('nice', 0.43366366624832153),
 ('compact', 0.43015509843826294)]

<h4> Adjective Intensity </h4>

In [303]:
Potts_list = pd.read_csv('wn-asr-multilevel-assess.csv')


Potts_list = Potts_list[Potts_list['Word'].apply(lambda x:x.split('/')[1]) == 'a']
Potts_list = Potts_list[['Word','Predicted','RawScore']]
Potts_list['Word'] = Potts_list['Word'].apply(lambda x:x.split('/')[0])
Potts_list.reset_index().drop('index',axis = 1)
# for i in Potts_list['Word']:
#     if 'tiny' in Potts_list['Word']:
#         print i
# 'tiny' isin Potts_list['Word']

# lol = [1,2,4]
# 1 in lol
Potts_list[Potts_list['Word'].isin(['square', 'nice', 'great', 'short'])]

adj = Potts_list['Word'].tolist()
adj

['100th',
 '10th',
 '11th',
 '12th',
 '13th',
 '14th',
 '15th',
 '16th',
 '17th',
 '18th',
 '19th',
 '20th',
 '21st',
 '23rd',
 '25th',
 '27th',
 '29th',
 '2d',
 '2nd',
 '3-dimensional',
 '30th',
 '35th',
 '38th',
 '39th',
 '3rd',
 '40th',
 '45th',
 '49th',
 '4th',
 '50th',
 '5th',
 '60th',
 '6th',
 '75th',
 '7th',
 '8th',
 'a couple of',
 'a few',
 'a la carte',
 'a priori',
 'abbreviated',
 'abhorrent',
 'able-bodied',
 'able',
 'abnormal',
 'abominable',
 'above-mentioned',
 'above',
 'abrasive',
 'abridged',
 'abrupt',
 'absent',
 'absolute',
 'absorbing',
 'abstract',
 'absurd',
 'abundant',
 'abused',
 'abusive',
 'abuzz',
 'abysmal',
 'academic',
 'accented',
 'acceptable',
 'accessible',
 'accessory',
 'accidental',
 'accompanying',
 'accomplished',
 'accountable',
 'accurate',
 'accusatory',
 'accustomed',
 'acerbic',
 'acid',
 'acidic',
 'acknowledged',
 'acrobatic',
 'acting',
 'active',
 'actual',
 'acute',
 'ad hoc',
 'adamant',
 'adaptable',
 'adaptive',
 'addictive',
 'a

In [305]:
def gen_rawscore(x):
    hits = [i for i in x.split() if i in adj]
    return Potts_list[Potts_list['Word'].isin(hits)]['RawScore'].sum()


train_exp = alldata[~alldata['Is_Response'].isnull()]
train_exp['score'] = train_exp['Description'].apply(lambda x:gen_rawscore(x))

/Users/prudhviraj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [327]:
test_exp = alldata[alldata['Is_Response'].isnull()]
test_exp['score']= test_exp['Description'].apply(lambda x:gen_rawscore(x))

/Users/prudhviraj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [328]:
test_exp

,User_ID,Description,Browser_Used,Device_Used,Is_Response,score
38932,id80132,looking motel close proximity tv taping dr phi...,2,1,NaN,-1.148672
38933,id80133,walking distance madison square garden penn su...,6,0,NaN,3.414922
38934,id80134,visited seattle business spent nights vintage ...,4,2,NaN,4.180653
38935,id80135,hotel location excellent rooms clean suite one...,1,1,NaN,5.899560
38936,id80136,hotel awesome love service anthony really grea...,7,1,NaN,8.158987
38937,id80137,rooms concierge services party scene great rig...,6,2,NaN,2.451855
38938,id80138,pleasant place stay hotel beautiful modern sty...,9,2,NaN,12.698381
38939,id80139,booked day arrived cost fridge cofeemaker room...,1,0,NaN,-1.767388
38940,id80140,unlike hotels grimace show family rather solo ...,3,2,NaN,0.734656
38941,id80141,hotel sorella birthday absolutely love hotel d...,3,1,NaN,2.645557


In [306]:
train_exp

,User_ID,Description,Browser_Used,Device_Used,Is_Response,score
0,id10326,room kind clean strong smell dogs generally av...,1,1,not happy,0.681378
1,id10327,stayed crown plaza april april staff friendly ...,5,1,not happy,7.041155
2,id10328,booked hotel hotwire lowest price could find g...,7,2,not happy,-4.724291
3,id10329,stayed husband sons way alaska cruise loved ho...,6,0,happy,10.313841
4,id10330,girlfriends stayed celebrate th birthdays plan...,1,2,not happy,-1.532165
5,id10331,rooms one nice clearly updated recently clean ...,6,0,happy,1.028412
6,id10332,husband stayed hotel times though fanciest hot...,2,2,not happy,0.146774
7,id10333,wife stayed glorious city back sf expensive fo...,3,1,happy,14.757629
8,id10334,boyfriend stayed fairmont recent trip san fran...,5,0,happy,6.823141
9,id10335,wonderful staff great location definately pric...,0,2,not happy,-0.519935


In [323]:
# num= 32170
#df1 = df[(df.a != -1) & (df.b != -1)]
len1 = len(train_exp[(train_exp['score'] > 0) & (train_exp['Is_Response'] == 'happy')])
len2 = len(train_exp[(train_exp['score'] < 0) & (train_exp['Is_Response'] == 'not happy')])
leno = len1 + len2
# print float (25674/32170)
# train_exp[train_exp['score'] > 0]
print leno

31579


In [292]:
alldata['Is_Response'][12]

'not happy'

In [293]:
# des = 'stayed crown plaza april april staff friendly attentive elevators tiny food restaurant delicious priced little high side course washington dc pool little children room fifth floor two comfortable beds plenty space one person tv little small todays standards limited number channels small bit mold bathtub area could removed little bleach appeared carpets vacummed every day reported light bulb burned never replaced ice machines odd numbered floors one floor work encountered staff elevator one evening mentioned ice machine severel hours later maid appeared door ice two mints im sure knew room little unnerving would stay business would come vacation'
# for i in desc.split():
#     print i
hits = [i for i in alldata['Description'][12].split() if i in adj]
Potts_list[Potts_list['Word'].isin(hits)]['RawScore'].sum()
Potts_list[Potts_list['Word'].isin(hits)]

,Word,Predicted,RawScore
263,amazing,positive,2.174223
501,available,neutral,0.077502
523,back,negative,-0.246684
577,best,positive,1.049428
581,better,negative,-0.426617
663,bright,neutral,-0.062167
738,careful,positive,0.552825
767,central,positive,0.860572
857,close,positive,0.568091
1286,disappointed,negative,-1.809954


In [294]:
alldata

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,room kind clean strong smell dogs generally av...,1,1,not happy
1,id10327,stayed crown plaza april april staff friendly ...,5,1,not happy
2,id10328,booked hotel hotwire lowest price could find g...,7,2,not happy
3,id10329,stayed husband sons way alaska cruise loved ho...,6,0,happy
4,id10330,girlfriends stayed celebrate th birthdays plan...,1,2,not happy
5,id10331,rooms one nice clearly updated recently clean ...,6,0,happy
6,id10332,husband stayed hotel times though fanciest hot...,2,2,not happy
7,id10333,wife stayed glorious city back sf expensive fo...,3,1,happy
8,id10334,boyfriend stayed fairmont recent trip san fran...,5,0,happy
9,id10335,wonderful staff great location definately pric...,0,2,not happy


True

In [129]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")


In [130]:
# model.wv['stock']
# set(model.wv.vocab)
model.wv['stock']
model['stock']

array([  3.03505100e-02,   1.60596400e-01,  -1.11081138e-01,
        -1.63804591e-02,  -1.48735508e-01,  -2.65642386e-02,
        -2.21956172e-03,  -5.09199910e-02,  -1.57651417e-02,
        -1.47938356e-02,  -4.41798232e-02,   3.98163386e-02,
         3.13495882e-02,  -1.33617863e-01,   1.60942733e-01,
         2.10653692e-02,   3.62939611e-02,   2.69761272e-02,
        -2.99033839e-02,  -6.22549951e-02,  -3.59230898e-02,
        -3.73910032e-02,   3.91953699e-02,   2.02494543e-02,
        -1.07907495e-02,   1.24959406e-02,   4.68926430e-02,
        -9.91849303e-02,  -1.18022703e-01,  -1.52901094e-02,
        -3.81200351e-02,  -1.31206159e-02,   9.93648469e-02,
        -3.83094661e-02,  -7.21385144e-03,  -6.11223020e-02,
        -8.87590125e-02,  -3.45392451e-02,   2.42315922e-02,
        -8.38157535e-02,   6.05082139e-02,   3.89160588e-02,
        -6.01166710e-02,   1.47270004e-03,   2.27981480e-03,
        -4.03908789e-02,  -3.00950203e-02,  -1.96806733e-02,
         4.58164439e-02,

In [131]:
# vocab_model_list = []
# vocab_model = (model.wv.vocab)
# for i in vocab_model:
#     vocab_model_list.append(i)
# print len(vocab_model)

vocab_model_list = [i for i in model.wv.vocab]
print len(vocab_model_list)

6782


In [132]:
review_train = for_w2v_train.tolist()
review_test = for_w2v_test.tolist()

review_train

[['room',
  'kind',
  'clean',
  'strong',
  'smell',
  'dogs',
  'generally',
  'average',
  'ok',
  'overnight',
  'stay',
  'youre',
  'fussy',
  'would',
  'consider',
  'staying',
  'price',
  'right',
  'breakfast',
  'free',
  'better',
  'nothing'],
 ['stayed',
  'crown',
  'plaza',
  'april',
  'april',
  'staff',
  'friendly',
  'attentive',
  'elevators',
  'tiny',
  'food',
  'restaurant',
  'delicious',
  'priced',
  'little',
  'high',
  'side',
  'course',
  'washington',
  'dc',
  'pool',
  'little',
  'children',
  'room',
  'fifth',
  'floor',
  'two',
  'comfortable',
  'beds',
  'plenty',
  'space',
  'one',
  'person',
  'tv',
  'little',
  'small',
  'todays',
  'standards',
  'limited',
  'number',
  'channels',
  'small',
  'bit',
  'mold',
  'bathtub',
  'area',
  'could',
  'removed',
  'little',
  'bleach',
  'appeared',
  'carpets',
  'vacummed',
  'every',
  'day',
  'reported',
  'light',
  'bulb',
  'burned',
  'never',
  'replaced',
  'ice',
  'machines'

In [133]:
import numpy as np

def makeFeatureVec(words,model,num_features):
    featureVec = np.zeros((num_features,),dtype='float32')
    n_words = 0
    
    vocab_model_list = [i for i in model.wv.vocab]
    for word in words:
        if word in vocab_model_list:
            n_words += 1
            featureVec = np.add(featureVec,model[word])
    
    featureVec = np.divide(featureVec,n_words)
    
    
    return featureVec 

def getAvgFeatureVecs(reviews,model,num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype='float32')
    
    for review in reviews:
        # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))
       # 
        reviewFeatureVecs[counter] = makeFeatureVec(review,model,num_features)
        counter += 1
    return reviewFeatureVecs
    

# print makeFeatureVec(review_train[1],model,num_features)

trainDataVecs =  getAvgFeatureVecs(review_train,model,num_features)

Review 0 of 38932
Review 1000 of 38932
Review 2000 of 38932
Review 3000 of 38932
Review 4000 of 38932
Review 5000 of 38932
Review 6000 of 38932
Review 7000 of 38932
Review 8000 of 38932
Review 9000 of 38932
Review 10000 of 38932
Review 11000 of 38932
Review 12000 of 38932
Review 13000 of 38932
Review 14000 of 38932
Review 15000 of 38932
Review 16000 of 38932
Review 17000 of 38932
Review 18000 of 38932
Review 19000 of 38932
Review 20000 of 38932
Review 21000 of 38932
Review 22000 of 38932
Review 23000 of 38932
Review 24000 of 38932
Review 25000 of 38932
Review 26000 of 38932
Review 27000 of 38932
Review 28000 of 38932
Review 29000 of 38932
Review 30000 of 38932
Review 31000 of 38932
Review 32000 of 38932
Review 33000 of 38932
Review 34000 of 38932
Review 35000 of 38932
Review 36000 of 38932
Review 37000 of 38932
Review 38000 of 38932


In [134]:
trainDataVecs =  getAvgFeatureVecs(review_train,model,num_features)


Review 0 of 38932
Review 1000 of 38932
Review 2000 of 38932
Review 3000 of 38932
Review 4000 of 38932
Review 5000 of 38932
Review 6000 of 38932
Review 7000 of 38932
Review 8000 of 38932
Review 9000 of 38932
Review 10000 of 38932
Review 11000 of 38932
Review 12000 of 38932
Review 13000 of 38932
Review 14000 of 38932
Review 15000 of 38932
Review 16000 of 38932
Review 17000 of 38932
Review 18000 of 38932
Review 19000 of 38932
Review 20000 of 38932
Review 21000 of 38932
Review 22000 of 38932
Review 23000 of 38932
Review 24000 of 38932
Review 25000 of 38932
Review 26000 of 38932
Review 27000 of 38932
Review 28000 of 38932
Review 29000 of 38932
Review 30000 of 38932
Review 31000 of 38932
Review 32000 of 38932
Review 33000 of 38932
Review 34000 of 38932
Review 35000 of 38932
Review 36000 of 38932
Review 37000 of 38932
Review 38000 of 38932


In [135]:
testDataVecs = getAvgFeatureVecs(review_test,model,num_features)

Review 0 of 29404
Review 1000 of 29404
Review 2000 of 29404
Review 3000 of 29404
Review 4000 of 29404
Review 5000 of 29404
Review 6000 of 29404
Review 7000 of 29404
Review 8000 of 29404
Review 9000 of 29404
Review 10000 of 29404
Review 11000 of 29404
Review 12000 of 29404
Review 13000 of 29404
Review 14000 of 29404
Review 15000 of 29404
Review 16000 of 29404
Review 17000 of 29404
Review 18000 of 29404
Review 19000 of 29404
Review 20000 of 29404
Review 21000 of 29404
Review 22000 of 29404
Review 23000 of 29404
Review 24000 of 29404
Review 25000 of 29404
Review 26000 of 29404
Review 27000 of 29404
Review 28000 of 29404
Review 29000 of 29404


In [136]:
# initialise the functions - we'll create separate models for each type.
countvec = CountVectorizer(analyzer='word', ngram_range = (1,2), min_df=150, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', ngram_range = (1,2), min_df = 150, max_features=500)

In [137]:
# create features
bagofwords = countvec.fit_transform(alldata['Description'])
tfidfdata = tfidfvec.fit_transform(alldata['Description'])


In [138]:
print tfidfdata

  (0, 365)	0.115949379735
  (0, 229)	0.347362124439
  (0, 76)	0.169044734282
  (0, 27)	0.360371547592
  (0, 304)	0.324153752757
  (0, 407)	0.143145890103
  (0, 499)	0.340591282081
  (0, 490)	0.161015281399
  (0, 411)	0.268655510748
  (0, 334)	0.24656697264
  (0, 364)	0.239913099862
  (0, 50)	0.189257697578
  (0, 169)	0.228128898771
  (0, 42)	0.265049916519
  (0, 298)	0.302183219349
  (1, 365)	0.0983868942336
  (1, 407)	0.0607320176345
  (1, 490)	0.136626806432
  (1, 409)	0.0701646556917
  (1, 401)	0.1199921226
  (1, 172)	0.083640488972
  (1, 129)	0.14986653891
  (1, 434)	0.157330118596
  (1, 165)	0.106873994691
  (1, 359)	0.104224985488
  :	:
  (68335, 380)	0.10872305095
  (68335, 35)	0.206724165266
  (68335, 176)	0.134306196989
  (68335, 471)	0.149554945841
  (68335, 112)	0.186389645241
  (68335, 421)	0.147036257173
  (68335, 63)	0.205561030489
  (68335, 293)	0.100933300591
  (68335, 351)	0.127316008148
  (68335, 186)	0.099192732899
  (68335, 399)	0.154141076807
  (68335, 414)	0.14017

In [139]:
print type(bagofwords)
print (bagofwords.shape
      )

<class 'scipy.sparse.csr.csr_matrix'>
(68336, 500)


In [140]:
bag_toarray = bagofwords.toarray()
print ((bag_toarray)[0])
print (bag_toarray)[1]
# vocab_tf = tfidfvec.get_feature_names()
# print (vocab_tf)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 

In [141]:

# Sum up the counts of each vocabulary word
# dist = np.sum(bag_toarray, axis=0)
# # print (dist)
# # For each, print the vocabulary word and the number of times it 
# # appears in the training set
# for tag, count in zip(vocab, dist):
#     print count, tag

In [142]:
alldata.head()

#label encode categorical features in data given
cols = ['Browser_Used','Device_Used']

for x in cols:
    lbl = LabelEncoder()
    alldata[x] = lbl.fit_transform(alldata[x])

alldata

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,room kind clean strong smell dogs generally av...,1,1,not happy
1,id10327,stayed crown plaza april april staff friendly ...,5,1,not happy
2,id10328,booked hotel hotwire lowest price could find g...,7,2,not happy
3,id10329,stayed husband sons way alaska cruise loved ho...,6,0,happy
4,id10330,girlfriends stayed celebrate th birthdays plan...,1,2,not happy
5,id10331,rooms one nice clearly updated recently clean ...,6,0,happy
6,id10332,husband stayed hotel times though fanciest hot...,2,2,not happy
7,id10333,wife stayed glorious city back sf expensive fo...,3,1,happy
8,id10334,boyfriend stayed fairmont recent trip san fran...,5,0,happy
9,id10335,wonderful staff great location definately pric...,0,2,not happy


In [143]:
# create dataframe for features
bow_df = pd.DataFrame(bagofwords.todense())
tfidf_df = pd.DataFrame(tfidfdata.todense())

In [144]:
bow_df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
tfidf_df.columns = ['col' + str(x) for x in tfidf_df.columns]

In [146]:
bow_df.columns

Index([u'col0', u'col1', u'col2', u'col3', u'col4', u'col5', u'col6', u'col7',
       u'col8', u'col9',
       ...
       u'col490', u'col491', u'col492', u'col493', u'col494', u'col495',
       u'col496', u'col497', u'col498', u'col499'],
      dtype='object', length=500)

In [147]:
# create separate data frame for bag of words and tf-idf

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

tfid_df_train = tfidf_df[:len(train)]
tfid_df_test = tfidf_df[len(train):]

In [148]:
bow_df_train

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
tfid_df_train

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.161015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.340591
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.136627,0.000000,0.000000,0.136304,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.101344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.186621,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.158929,0.000000,0.111832,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.088210,0.000000,0.000000,0.176004,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.068236,0.000000,0.144045,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.207843
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.242826,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [150]:
# split the merged data file into train and test respectively
train_feats = alldata[~pd.isnull(alldata.Is_Response)]
test_feats = alldata[pd.isnull(alldata.Is_Response)]

In [151]:
train_feats.columns

Index([u'User_ID', u'Description', u'Browser_Used', u'Device_Used',
       u'Is_Response'],
      dtype='object')

In [152]:
train_feats

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,room kind clean strong smell dogs generally av...,1,1,not happy
1,id10327,stayed crown plaza april april staff friendly ...,5,1,not happy
2,id10328,booked hotel hotwire lowest price could find g...,7,2,not happy
3,id10329,stayed husband sons way alaska cruise loved ho...,6,0,happy
4,id10330,girlfriends stayed celebrate th birthdays plan...,1,2,not happy
5,id10331,rooms one nice clearly updated recently clean ...,6,0,happy
6,id10332,husband stayed hotel times though fanciest hot...,2,2,not happy
7,id10333,wife stayed glorious city back sf expensive fo...,3,1,happy
8,id10334,boyfriend stayed fairmont recent trip san fran...,5,0,happy
9,id10335,wonderful staff great location definately pric...,0,2,not happy


In [153]:
test_feats

,User_ID,Description,Browser_Used,Device_Used,Is_Response
38932,id80132,looking motel close proximity tv taping dr phi...,2,1,NaN
38933,id80133,walking distance madison square garden penn su...,6,0,NaN
38934,id80134,visited seattle business spent nights vintage ...,4,2,NaN
38935,id80135,hotel location excellent rooms clean suite one...,1,1,NaN
38936,id80136,hotel awesome love service anthony really grea...,7,1,NaN
38937,id80137,rooms concierge services party scene great rig...,6,2,NaN
38938,id80138,pleasant place stay hotel beautiful modern sty...,9,2,NaN
38939,id80139,booked day arrived cost fridge cofeemaker room...,1,0,NaN
38940,id80140,unlike hotels grimace show family rather solo ...,3,2,NaN
38941,id80141,hotel sorella birthday absolutely love hotel d...,3,1,NaN


In [154]:
### set target variable

train_feats['Is_Response'] = [1 if x == 'happy' else 0 for x in train_feats['Is_Response']]

/Users/prudhviraj/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [155]:
train_feats.head

<bound method DataFrame.head of        User_ID                                        Description  \
0      id10326  room kind clean strong smell dogs generally av...   
1      id10327  stayed crown plaza april april staff friendly ...   
2      id10328  booked hotel hotwire lowest price could find g...   
3      id10329  stayed husband sons way alaska cruise loved ho...   
4      id10330  girlfriends stayed celebrate th birthdays plan...   
5      id10331  rooms one nice clearly updated recently clean ...   
6      id10332  husband stayed hotel times though fanciest hot...   
7      id10333  wife stayed glorious city back sf expensive fo...   
8      id10334  boyfriend stayed fairmont recent trip san fran...   
9      id10335  wonderful staff great location definately pric...   
10     id10336  steps times square nice rooms stayed nights gr...   
11     id10337  wife kids stayed valentines weekend really nic...   
12     id10338  stay jolly madison xmas period main feature lo...   
13

In [156]:
# merge count (bag of word) features into train
train_feats1 = pd.concat([train_feats[cols], bow_df_train], axis = 1)
test_feats1 = pd.concat([test_feats[cols], bow_df_test], axis=1)

test_feats1.reset_index(drop=True, inplace=True)
bow_df_test

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
38932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
38935,0,0,0,0,0,0,3,0,0,0,...,3,0,0,0,0,0,0,0,0,0
38936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38937,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38938,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
38939,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
38940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38941,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
word2vec_df_train = pd.DataFrame(trainDataVecs)
word2vec_df_test = pd.DataFrame(testDataVecs)

word2vec_df_test.index += 38932


train_feats3 = pd.concat([train_feats[cols],word2vec_df_train], axis = 1)
test_feats3 = pd.concat([test_feats[cols],word2vec_df_test], axis = 1)
# test_feats3.reset_index(drop=True, inplace=True)
test_feats3

,Browser_Used,Device_Used,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
38932,2,1,0.006514,0.013842,-0.014869,0.004860,-0.019351,0.002960,-0.001652,-0.008972,...,-0.002076,0.010560,-0.000876,0.011650,-0.011971,-0.008756,0.017330,0.001307,0.013911,0.026871
38933,6,0,-0.001560,-0.006517,0.018007,0.026437,0.000604,0.005602,0.000411,0.016337,...,0.018289,0.002359,-0.024888,-0.023777,-0.005986,-0.008945,0.027627,-0.012935,-0.002729,0.016361
38934,4,2,-0.005738,-0.011614,0.016988,-0.001691,-0.004764,-0.010785,-0.006360,-0.005318,...,0.001598,-0.000223,0.006457,0.018918,0.005388,0.009496,-0.014165,-0.010858,-0.002316,-0.009288
38935,1,1,0.007181,0.019357,0.009056,0.011116,0.007935,0.015303,-0.006153,-0.000206,...,-0.017849,0.014295,-0.004982,0.019491,-0.008525,-0.013359,0.017290,-0.009548,-0.016394,-0.022317
38936,7,1,0.025027,0.013486,0.038597,-0.010714,-0.017635,-0.006008,-0.010978,-0.009178,...,0.014443,-0.014801,-0.008174,-0.014985,0.010120,0.015044,-0.020860,-0.047522,0.021530,0.023622
38937,6,2,0.008261,0.002866,0.008132,-0.013842,-0.016141,-0.007925,-0.012031,-0.022130,...,0.015389,-0.014543,-0.013352,-0.011041,-0.007216,0.005151,-0.012634,-0.020738,0.034393,0.008931
38938,9,2,0.000517,-0.005818,0.037304,-0.001080,-0.007573,0.004958,-0.003778,0.003168,...,-0.000815,0.003735,0.002344,0.011969,0.000329,0.001768,0.001661,-0.008752,0.005814,-0.012937
38939,1,0,-0.011070,-0.006764,0.028277,0.000856,-0.000880,0.004733,-0.019695,0.021282,...,-0.004907,0.001118,-0.003664,0.008927,-0.006941,-0.005238,0.005178,0.010505,-0.017453,-0.024761
38940,3,2,-0.006568,-0.003642,0.044974,0.000664,-0.015675,0.001144,-0.003560,0.011398,...,-0.013162,0.005672,-0.017040,0.024911,-0.001207,-0.007883,-0.015405,-0.013644,-0.003930,0.004812
38941,3,1,0.025847,-0.017146,0.014086,-0.007416,-0.016681,0.011585,-0.011782,-0.010739,...,0.034828,-0.004460,0.008273,-0.018424,-0.012807,-0.002008,-0.016530,-0.028393,0.022617,0.017317


In [158]:
train_feats1

,Browser_Used,Device_Used,col0,col1,col2,col3,col4,col5,col6,col7,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,5,1,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,0,0,0
2,7,2,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,0,0,0,0,0,0,0,0,...,3,0,1,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
6,2,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
type(test_feats1)

pandas.core.frame.DataFrame

In [160]:
# merge into a new data frame with tf-idf features
train_feats2 = pd.concat([train_feats[cols], tfid_df_train], axis=1)
test_feats2 = pd.concat([test_feats[cols], tfid_df_test], axis=1)


In [329]:
train_feats_adj = pd.concat([train_feats[cols], train_exp['score']], axis=1)
test_feats_adj = pd.concat([test_feats[cols], test_exp['score']], axis=1)


In [341]:
train_feats_adj

,Browser_Used,Device_Used,score
0,1,1,0.681378
1,5,1,7.041155
2,7,2,-4.724291
3,6,0,10.313841
4,1,2,-1.532165
5,6,0,1.028412
6,2,2,0.146774
7,3,1,14.757629
8,5,0,6.823141
9,0,2,-0.519935


In [161]:
train_feats_bow_tfid = pd.concat([train_feats[cols], bow_df_train,tfid_df_train],axis = 1)
test_feats_bow_tfid = pd.concat([test_feats[cols], bow_df_test,tfid_df_test],axis = 1)

In [162]:
test_feats_bow_tfid

,Browser_Used,Device_Used,col0,col1,col2,col3,col4,col5,col6,col7,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
38932,2,1,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38933,6,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38934,4,2,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.163345,0.000000,0.000000
38935,1,1,0,0,0,0,0,0,3,0,...,0.217578,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38936,7,1,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38937,6,2,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38938,9,2,0,0,0,0,0,0,0,0,...,0.057156,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38939,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38940,3,2,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
38941,3,1,0,1,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [163]:
train_feats_bow_tfid_word2vec = pd.concat([train_feats[cols], bow_df_train,tfid_df_train,word2vec_df_train],axis = 1)
test_feats_bow_tfid_word2vec = pd.concat([test_feats[cols], bow_df_test,tfid_df_test,word2vec_df_test],axis = 1)

In [332]:
train_feats_bow_tfid_word2vec_adj = pd.concat([train_feats[cols], bow_df_train,tfid_df_train,word2vec_df_train,train_exp['score']],axis = 1)
test_feats_bow_tfid_word2vec_adj = pd.concat([test_feats[cols], bow_df_test,tfid_df_test,word2vec_df_test,test_exp['score']],axis = 1)

In [164]:
test_feats_bow_tfid_word2vec

,Browser_Used,Device_Used,col0,col1,col2,col3,col4,col5,col6,col7,...,290,291,292,293,294,295,296,297,298,299
38932,2,1,0,0,0,0,0,0,0,0,...,-0.002076,0.010560,-0.000876,0.011650,-0.011971,-0.008756,0.017330,0.001307,0.013911,0.026871
38933,6,0,0,0,0,0,0,0,0,0,...,0.018289,0.002359,-0.024888,-0.023777,-0.005986,-0.008945,0.027627,-0.012935,-0.002729,0.016361
38934,4,2,0,0,0,0,0,0,0,0,...,0.001598,-0.000223,0.006457,0.018918,0.005388,0.009496,-0.014165,-0.010858,-0.002316,-0.009288
38935,1,1,0,0,0,0,0,0,3,0,...,-0.017849,0.014295,-0.004982,0.019491,-0.008525,-0.013359,0.017290,-0.009548,-0.016394,-0.022317
38936,7,1,0,0,0,0,0,0,0,0,...,0.014443,-0.014801,-0.008174,-0.014985,0.010120,0.015044,-0.020860,-0.047522,0.021530,0.023622
38937,6,2,0,0,0,0,0,0,0,0,...,0.015389,-0.014543,-0.013352,-0.011041,-0.007216,0.005151,-0.012634,-0.020738,0.034393,0.008931
38938,9,2,0,0,0,0,0,0,0,0,...,-0.000815,0.003735,0.002344,0.011969,0.000329,0.001768,0.001661,-0.008752,0.005814,-0.012937
38939,1,0,0,0,0,0,0,0,0,0,...,-0.004907,0.001118,-0.003664,0.008927,-0.006941,-0.005238,0.005178,0.010505,-0.017453,-0.024761
38940,3,2,0,0,0,0,0,0,0,0,...,-0.013162,0.005672,-0.017040,0.024911,-0.001207,-0.007883,-0.015405,-0.013644,-0.003930,0.004812
38941,3,1,0,1,0,0,0,0,0,0,...,0.034828,-0.004460,0.008273,-0.018424,-0.012807,-0.002008,-0.016530,-0.028393,0.022617,0.017317


In [165]:
train_feats_bow_tfid_word2vec .shape

(38932, 1302)

In [166]:
train_feats_bow_tfid

,Browser_Used,Device_Used,col0,col1,col2,col3,col4,col5,col6,col7,...,col490,col491,col492,col493,col494,col495,col496,col497,col498,col499
0,1,1,0,0,0,0,0,0,0,0,...,0.161015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.340591
1,5,1,0,0,0,0,0,0,0,0,...,0.136627,0.000000,0.000000,0.136304,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,7,2,0,0,0,0,0,0,0,0,...,0.101344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,6,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,1,2,0,0,0,0,0,0,0,0,...,0.158929,0.000000,0.111832,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,6,0,0,0,0,0,0,0,0,0,...,0.088210,0.000000,0.000000,0.176004,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,2,2,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,3,1,0,0,0,0,0,0,0,0,...,0.068236,0.000000,0.144045,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,5,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.207843
9,0,2,0,0,0,0,0,1,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [167]:
# let's check cross validation score of the model
# cv score acts a unbiased estimate of models accuracy on unseen data

mod1 = GaussianNB()
target = train_feats['Is_Response']

In [168]:
target

0        0
1        0
2        0
3        1
4        0
5        1
6        0
7        1
8        1
9        0
10       1
11       1
12       0
13       1
14       1
15       1
16       0
17       0
18       1
19       0
20       0
21       1
22       0
23       1
24       1
25       0
26       1
27       1
28       1
29       0
        ..
38902    1
38903    1
38904    1
38905    0
38906    1
38907    1
38908    1
38909    0
38910    1
38911    1
38912    1
38913    0
38914    0
38915    1
38916    1
38917    1
38918    0
38919    1
38920    1
38921    1
38922    0
38923    0
38924    1
38925    1
38926    0
38927    1
38928    0
38929    0
38930    0
38931    1
Name: Is_Response, dtype: int64

In [169]:
## Naive Bayes 1
print(cross_val_score(mod1, train_feats1, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.78389831  0.77356794  0.77536604  0.78037503  0.78474184]


In [170]:
## Naive Bayes 2 - tfidf is giving higher CV score
print(cross_val_score(mod1, train_feats2, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.79917822  0.80272284  0.7968148   0.80824557  0.79822759]


In [171]:
## Naive Bayes 3
print(cross_val_score(mod1, train_feats_bow_tfid, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.8029019   0.80195222  0.80452094  0.80747496  0.80490624]


In [172]:
## Naive Bayes 4 -word2vec features
print(cross_val_score(mod1, train_feats3, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.80880842  0.81107115  0.81119959  0.81389674  0.80811713]


In [173]:
## Naive Bayes 5 -bow+tfid+word2vec
print(cross_val_score(mod1,  train_feats_bow_tfid_word2vec, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.8174114   0.81864886  0.81646545  0.82083226  0.81967634]


In [331]:
## Naive Bayes 6 adj
print(cross_val_score(mod1,  train_feats_adj, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.82241911  0.82198818  0.82481377  0.82430003  0.82404315]


In [333]:
## Naive Bayes 7 -bow+tfid+word2vec+adj
print(cross_val_score(mod1,  train_feats_bow_tfid_word2vec_adj, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.81766821  0.82070383  0.81723606  0.82224506  0.82031852]


In [339]:
# make our first set of predictions

clf1 = GaussianNB()
clf1.fit(train_feats1, target)

clf2 = GaussianNB()
clf2.fit(train_feats2, target)

clf3 = GaussianNB()
clf3.fit(train_feats_bow_tfid_word2vec,target)

clf4 = GaussianNB()
clf4.fit(train_feats_bow_tfid_word2vec_adj,target)

clf5 = GaussianNB()
clf5.fit(train_feats_adj,target)



GaussianNB(priors=None)

In [ ]:
from sklearn import svm
clf_svm = svm.SVC(kernel='linear')
print(cross_val_score(clf_svm, train_feats1, target, cv=5, scoring=make_scorer(accuracy_score)))

In [343]:
preds1 = clf1.predict(test_feats1)
preds2 = clf2.predict(test_feats2)
preds3 =  clf3.predict(test_feats_bow_tfid_word2vec)
preds4 =  clf4.predict(test_feats_bow_tfid_word2vec_adj)
preds5 =  clf5.predict(test_feats_adj)


# test_feats_bow_tfid_word2vec

In [351]:
preds6 = [1 if i > 0 else 0 for i in test_feats_adj['score']]

In [176]:
def to_labels(x):
    if x == 1:
        return "happy"
    return "not_happy"

In [177]:
sub1 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds1})
sub1['Is_Response'] = sub1['Is_Response'].map(lambda x: to_labels(x))

In [178]:
sub2 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds2})
sub2['Is_Response'] = sub2['Is_Response'].map(lambda x: to_labels(x))

In [179]:
sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds3})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))

In [336]:
sub4 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds4})
sub4['Is_Response'] = sub4['Is_Response'].map(lambda x: to_labels(x))

In [344]:
sub5 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds5})
sub5['Is_Response'] = sub5['Is_Response'].map(lambda x: to_labels(x))

In [352]:
sub6 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds6})
sub6['Is_Response'] = sub6['Is_Response'].map(lambda x: to_labels(x))

In [345]:
sub1 = sub1[['User_ID', 'Is_Response']]
sub2 = sub2[['User_ID', 'Is_Response']]
sub3 = sub3[['User_ID', 'Is_Response']]
sub4 = sub4[['User_ID', 'Is_Response']]
sub5 = sub5[['User_ID', 'Is_Response']]


In [353]:
sub6 = sub6[['User_ID', 'Is_Response']]

In [349]:
## write submission files
sub1.to_csv('sub1_cv.csv', index=False)  # accuracy = 77.27
sub2.to_csv('sub2_tf.csv', index=False)  #accuracy =  79.805
sub3.to_csv('sub3_tf.csv', index=False)  # accuracy = 81.28


In [348]:
sub4.to_csv('sub4_tf.csv', index=False) #accuracy = 81.36

In [346]:
sub5.to_csv('sub5_tf.csv', index=False) #accuracy = 82.462

In [354]:
sub6.to_csv('sub6_tf.csv', index=False) #accuracy = 80.928